In [29]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_gen = ImageDataGenerator(rescale =(1/255.))
train_dir = "./training_set/"
test_dir = "./test_set/"

In [31]:
train_gen = image_gen.flow_from_directory(directory = train_dir,
                                          batch_size=32,
                                          target_size = (224,224),
                                          classes=['cats','dogs'],
                                          class_mode='binary',
                                          seed = 2022)
val_gen = image_gen.flow_from_directory(test_dir,
                                         batch_size=32,
                                         target_size = (224,224),
                                         classes=['cats','dogs'],
                                         class_mode='binary',
                                         seed = 2022)

                                 

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [32]:
class CNN(tf.keras.Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.convolutions1 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(32,(3,3),padding='same', input_shape=(28,28,3)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2)),
            
        ])
        self.convolutions2 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64,(3,3),padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2))
        ])
        self.convolutions3 = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64,(3,3),padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.MaxPool2D((2,2))
        ])
        
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(256),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation= 'sigmoid')
        ])
    def call(self,x):
        x = self.convolutions1(x)
        x = self.convolutions2(x)
        x = self.convolutions3(x)
        x = self.classifier(x)
        return x
    
        

In [33]:
model = CNN()
model.build(input_shape=(128,28,28,3))

In [34]:
model.summary()

Model: "cnn_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_40 (Sequential)   (None, 14, 14, 32)        1024      
_________________________________________________________________
sequential_41 (Sequential)   multiple                  18752     
_________________________________________________________________
sequential_42 (Sequential)   multiple                  37184     
_________________________________________________________________
sequential_43 (Sequential)   multiple                  147969    
Total params: 204,929
Trainable params: 204,609
Non-trainable params: 320
_________________________________________________________________


In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.01)
def schedule(epoch, lr):
    if epoch <5:
        return lr 
    elif epoch < 10:
        return lr*tf.math.exp(-0.1)
    else:
        return lr*tf.math.exp(-0.01)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(schedule)
earlystopping = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='tmp_checkpoint.ckpt',
                                                save_weights_only=True,
                                                save_best_only=True,
                                                monitor = 'val_loss',
                                                verbose = 1)

In [36]:
model.compile(optimizer=optimizer, loss = tf.losses.BinaryCrossentropy(from_logits=True), metrics = ['accuracy'])

In [38]:
history = model.fit(train_gen, callbacks=[lr_schedule, earlystopping, checkpoint],validation_data=val_gen, epochs = 20,batch_size=128)

ValueError: The `batch_size` argument must not be specified for the given input type. Received input: <keras_preprocessing.image.directory_iterator.DirectoryIterator object at 0x0000027900375080>, batch_size: 128